下载高德地图行政区划数据，制作 shapefile 文件。

In [ ]:
import json
from io import StringIO

import requests
import shapefile
import shapely.geometry as sgeom
from shapely.ops import unary_union
import numpy as np
from frykit import DATA_DIRPATH

高德 web API 查询行政区域：[https://lbs.amap.com/api/webservice/guide/api/district](https://lbs.amap.com/api/webservice/guide/api/district)

按入门指南注册并申请密钥（key），拼接 HTTP 请求的 URL，接收并解析返回的数据。

In [ ]:
key = '1145141919810'

特殊省名指这些省的子一级（即市级）就是它们自身。

In [ ]:
province_names = ['北京市', '天津市', '河北省', '山西省', '内蒙古自治区', '辽宁省', '吉林省', '黑龙江省', '上海市', '江苏省', '浙江省', '安徽省', '福建省', '江西省', '山东省', '河南省', '湖北省', '湖南省', '广东省', '广西壮族自治区', '海南省', '重庆市', '四川省', '贵州省', '云南省', '西藏自治区', '陕西省', '甘肃省', '青海省', '宁夏回族自治区', '新疆维吾尔自治区', '台湾省', '香港特别行政区', '澳门特别行政区']
special_province_names = ['北京市', '天津市', '上海市', '重庆市', '台湾省', '香港特别行政区', '澳门特别行政区']
len(province_names), len(special_province_names)

根据省名收集市名和 adcode。

收集方法是在 URL 里加入 `subdistrict=1` 以返回子区划，认为子区划就算是市级。

高德文档里的相关下载一栏可以下载到城市编码表，一开始的想法是根据表中 adcode 最后两位不为 00 来找出所有市。但这样会遗漏掉直辖县之类的区划，导致最后凑不出完整的中国地图。所以最后采用了上一段的方法。

In [ ]:
province_records = []
city_records = []
for province_name in province_names:
    url = f'https://restapi.amap.com/v3/config/district?key={key}&keywords={province_name}&subdistrict=1'
    response = requests.get(url)
    content = json.loads(response.content.decode())
    province_district = content['districts'][0]
    province_adcode = int(province_district['adcode'])
    province_records.append({
        'pr_name': province_name,
        'pr_adcode': province_adcode
    })

    if province_name in special_province_names:
        city_records.append({
            'ct_name': province_name,
            'ct_adcode': province_adcode,
            'pr_name': province_name,
            'pr_adcode': province_adcode
        })
        continue

    for district_data in province_district['districts']:
        city_district = district_data
        city_records.append({
            'ct_name': city_district['name'],
            'ct_adcode': int(city_district['adcode']),
            'pr_name': province_name,
            'pr_adcode': province_adcode
        })

In [ ]:
def create_prj_file(filepath):
    '''制作prj文件.'''
    wkt = 'GEOGCS["WGS84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.017453292519943295]]'
    with open(str(filepath), 'w', encoding='utf-8') as f:
        f.write(wkt)

In [ ]:
def polyline_to_coordinates(polyline):
    '''polyline字符串转为一组多边形的坐标序列.'''
    return [
         np.loadtxt(StringIO(part), delimiter=',').tolist()
         for part in polyline.replace(';', '\n').split('|')
    ]

In [ ]:
def polygon_to_coordinates(polygon):
    '''多边形对象转为shapefile适用的坐标序列.'''
    coordinates = []
    for polygon in getattr(polygon, 'geoms', [polygon]):
        for ring in [polygon.exterior, *polygon.interiors]:
            coordinates.append(ring.coords[:])

    return coordinates

`polyline` 用字符串表示多边形坐标序列。不同多边形用 `|` 分隔，不同点用 `;` 分割，xy 用 `,` 分隔。多边形环的绕行方向都是顺时针，那么问题来了，怎么判断多边形的洞，怎么判断 `MultiPolygon` 呢？

测试后大致发现，`polyline` 里不是用单独的多边形表示洞，而是将带洞的多边形切成两个独立的多边形，当这两个多边形拼在一起时，就会凑出一个洞。类似下图的效果。因此这里的策略是直接合并 `polyline` 里的所有多边形。

关于多边形绕行方向：

- shapely 里构造 `Polygon` 时方向无所谓，因为已经通过 `shell` 和 `holes` 参数明确指定了外环和内环。但经过运算后会变成外环顺时针内环逆时针。
- Shapefile 要求外环顺时针，内环逆时针；GeoJSON 要求外环逆时针，内环顺时针，不过并不强制。
- PyShp 的 `__geo_interface__` 接口能将 shapefile 转为 GeoJSON，但除了 2.2.0 版本外不会改变底层数据的绕行方向。
- `sgeom.shape` 会用到 `__geo_interface__` 接口。
- `mpath.Path` 要求内外环方向不一致即可。

总结：用 shapely 和 PyShp 处理全为顺时针的 `polyline` 数据，最后能得到外环顺时针，内环逆时针的 shapefile 文件。后续用于 Matplotlib 时能区分出洞。

In [ ]:
geom1 = sgeom.Polygon([(0, 0), (0, 3), (3, 3), (3, 2), (1, 2), (1, 1), (3, 1), (3, 0), (0, 0)])
geom2 = sgeom.Polygon([(2, 1), (2, 2), (3, 2), (3, 1), (2, 1)])
unary_union([geom1, geom2])

获取区划坐标需要在 URL 中加入 `extensions=all`。

某些省市存在飞地，会出现子多边形相隔很远的 `MultiPolygon`（例如天津市）。

In [ ]:
shp_dirpath = DATA_DIRPATH / 'shp'
border_filepath = shp_dirpath / 'cn_border.shp'
create_prj_file(shp_dirpath / f'{border_filepath.stem}.prj')
with shapefile.Writer(str(border_filepath), shapeType=5) as writer:
    writer.fields = [['cn_name', 'C', 80, 0], ['cn_adcode', 'N', 6, 0]]
    url = f'https://restapi.amap.com/v3/config/district?key={key}&keywords=100000&subdistrict=0&extensions=all'
    response = requests.get(url)
    content = json.loads(response.content.decode())
    polyline = content['districts'][0]['polyline']
    coordinates = polyline_to_coordinates(polyline)
    polygon = unary_union([sgeom.Polygon(coords) for coords in coordinates])
    coordinates = polygon_to_coordinates(polygon)
    writer.record(cn_name='中华人民共和国', cn_adcode=100000)
    writer.poly(coordinates)

In [ ]:
province_filepath = shp_dirpath / 'cn_province.shp'
create_prj_file(shp_dirpath / f'{province_filepath.stem}.prj')
with shapefile.Writer(str(province_filepath), shapeType=5) as writer:
    writer.fields = [['pr_name', 'C', 80, 0], ['pr_adcode', 'N', 6, 0]]
    for record in province_records:
        url = f'https://restapi.amap.com/v3/config/district?key={key}&keywords={record["pr_adcode"]}&subdistrict=0&extensions=all'
        response = requests.get(url)
        content = json.loads(response.content.decode())
        polyline = content['districts'][0]['polyline']
        coordinates = polyline_to_coordinates(polyline)
        polygon = unary_union([sgeom.Polygon(coords) for coords in coordinates])
        coordinates = polygon_to_coordinates(polygon)
        writer.record(**record)
        writer.poly(coordinates)
        print(record)

In [ ]:
city_filepath = shp_dirpath / 'cn_city.shp'
create_prj_file(shp_dirpath / f'{city_filepath.stem}.prj')
with shapefile.Writer(str(city_filepath), shapeType=5) as writer:
    writer.fields = [
        ['ct_name', 'C', 80, 0],
        ['ct_adcode', 'N', 6, 0],
        ['pr_name', 'C', 80, 0],
        ['pr_adcode', 'N', 6, 0]
    ]
    for record in city_records:
        url = f'https://restapi.amap.com/v3/config/district?key={key}&keywords={record["ct_adcode"]}&subdistrict=0&extensions=all'
        response = requests.get(url)
        content = json.loads(response.content.decode())
        polyline = content['districts'][0]['polyline']
        coordinates = polyline_to_coordinates(polyline)
        polygon = unary_union([sgeom.Polygon(coords) for coords in coordinates])
        coordinates = polygon_to_coordinates(polygon)
        writer.record(**record)
        writer.poly(coordinates)
        print(record)

[https://datav.aliyun.com/portal/school/atlas/area_selector](https://datav.aliyun.com/portal/school/atlas/area_selector) 全国的 GeoJSON 数据里含多边形表示的九段线，以此制作九段线的 shapefile 文件。注意 `writer.shape` 会自动将 GeoJSON 里逆时针的外环改为顺时针。

In [ ]:
nine_line_filepath = shp_dirpath / 'nine_line.shp'
create_prj_file(shp_dirpath / f'{nine_line_filepath.stem}.prj')
with shapefile.Writer(str(nine_line_filepath)) as writer:
    url = 'https://geo.datav.aliyun.com/areas_v3/bound/100000_full.json'
    response = requests.get(url)
    content = json.loads(response.content.decode())
    geometry = content['features'][-1]['geometry']
    writer.fields = [['cn_adcode', 'C', 80, 0], ['cn_name', 'C', 80, 0]]
    writer.record(cn_adcode='100000', cn_name='九段线')
    writer.shape(geometry)